# Project Title
### Data Engineering Capstone Project


In [1]:
# imports
import configparser
from datetime import datetime
import os
import glob
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import udf, col,concat_ws, from_unixtime, substring, monotonically_increasing_id,split, col,array, lit, explode, from_json, json_tuple, length, to_timestamp  
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format,from_unixtime
from pyspark.sql.types import StructType, StringType, ArrayType, TimestampType, IntegerType, DoubleType

### Step 1: Scope the Project and Gather Data

This is the final capstone project for the Data Engineer Nano Degree. Udacity provides
a topic of the final project with provided dataset, where students should show their gained 
through their learning path knowledge. Beside of provided topic for the capstone project, there is 
also a chance to choose your own dataset, but with some strict requirements:
>- _Dataset should contain at least 1 Million rows_
>- _Dataset should have at least two different data sources(JSON, CSV,...)_

In my case I've decided to explore my own dataset, which I found in Kaggle, called
  [The Movies Kaggle dataset.](https://www.kaggle.com/rounakbanik/the-movies-dataset?select=credits.csv)

### Goal of this project  
The goal of this project is to create an ETL process to extract,
transform and load data from existing CSV/JSON files from S3 bucket to AWS Redshift Datawarehouse
using Apache Airflow to automate as much as possible ETL process. Star Schema of tables in Redshift
allows quick analysys of data by using simple queries without JOIN statements. Apache Airflow allows
to schedule tasks and see the execution status of Pipeline steps. 
Data Warehouse has an ability to distribute data by key among CPU's which
encreases query the Data. 

#### Movie Dataset
This Dataset is taken from [The Movies Kaggle dataset.](https://www.kaggle.com/rounakbanik/the-movies-dataset?select=credits.csv)
These files contain metadata for all 45,000 movies listed in the Full MovieLens Dataset. The dataset consists of movies released on or before July 2017. Data points include cast, crew, plot keywords, budget, revenue, posters, release dates, languages, production companies, countries, TMDB vote counts and vote averages.
This dataset also has files containing 26 million ratings from 270,000 users for all 45,000 movies. Ratings are on a scale of 1-5 and have been obtained from the official GroupLens website.

### Purpose of final Dataset
With a help of final prepared dataset following analysis could be done:
- Predicting movie revenue or movie success
- What movies tend to get higher vote counts and vote averages
- Building content based and collaboration filtering based recommendation engines

### Source Files and Data Structure
>- movies_metadata.csv: 
> 
> The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

>- ratings.json
> 
> contains information about: ratings of movies given by users in particular time.

>- credits.csv
> 
> Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

>- keywords.csv:
> 
> Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

### Tools used in this project


- Amazon S3 - File Storage
- Amazon Redshift - for data analztics OLAP
- Apache Airflow - for building pipelines

These are open source tolls I chosen, because they are highly standartised and have huge popularity among developers, which provides enourmous amount of documentation sources for utilisition of these tools.

In [2]:
### Step 1.1 Setting Access to the Source files and reading them to explore

In [3]:
#Getting AWS credentials from config file
config = configparser.ConfigParser()
config.read('dl.cfg')

AWS_ACCESS_KEY_ID=config.get('default','AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY=config.get('default','AWS_SECRET_ACCESS_KEY')

In [4]:
# Create Spark Session
spark = SparkSession.builder \
            .appName("my_app") \
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
            .config("spark.sql.broadcastTimeout", "360000")\
            .getOrCreate()


In [5]:
#Setting AWS credentials to spark 
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", AWS_ACCESS_KEY_ID)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY)
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.BasicAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")


In [6]:
# Read data from source files to spark dataframes from S3 bucket

movies_df = spark.read.format('csv').options(header='true', inferSchema='true', mode='PERMISSIVE').load("s3a://tempbucket1168/movies_metadata.csv")
keywords_df = spark.read.format('csv').options(header='true', inferSchema='true', mode='PERMISSIVE').load("s3a://tempbucket1168/keywords.csv")
credits_df = spark.read.format('csv').options(header='true', inferSchema='true', mode='PERMISSIVE').load("s3a://tempbucket1168/credits.csv")
ratings_df =spark.read.format('org.apache.spark.sql.json').load("s3a://tempbucket1168/ratings.json")



### Step 2: Explore and Assess the Data


In [7]:
#### Explore the Data

In [8]:
## Exploring dataframe schemas

movies_df.printSchema()
keywords_df.printSchema()
credits_df.printSchema()
ratings_df.printSchema()

root
 |-- adult: string (nullable = true)
 |-- belongs_to_collection: string (nullable = true)
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- video: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nu

In [9]:
## Exploring data structures of movies_df
# For our data analysis Team we are interested in the following column extraction:
# id, title, overview, release_date, original_language, runtime, vote_average, production_companies, vote_count, revenue, budget
movies_df.head(1)

#Following quality issues:
# id->(Null values, not integer, duplicates);title->(Null values);overview->(Null values);release_date->(Null values, not date format);original_language->(Null values);
# runtime->(Null values, not integer);vote_average->(Null values, not float);production_companies->(Null values, not array);vote_count->(Null values, not float);
# revenue->(Null values, not float);budget->(Null values, not float)

[Row(adult='False', belongs_to_collection="{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}", budget='30000000', genres="[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]", homepage='http://toystory.disney.com/toy-story', id='862', imdb_id='tt0114709', original_language='en', original_title='Toy Story', overview="Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.", popularity='21.946943', poster_path='/rhIRbceoE9lR4veEXuwCC2wARtG.jpg', production_companies="[{'name': 'Pixar Animation Studios', 'id': 3}]", production_countries="[{'iso_3166_1': 'US', 'name': 'United States of America'}]", relea

In [10]:
## Exploring data structures of keywords_df
# For our data analysis Team we are interested in the following column extraction:
# id, keywords

keywords_df.head(1)

#Following quality issues:
# id->(Null values, not integer, duplicates);keywords->(Null values, not array)

[Row(id=862, keywords="[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]")]

In [11]:
## Exploring data structures of credits_df
# For our data analysis Team we are interested in the following column extraction:
# cast, crew, id

credits_df.head(1)

#Following quality issues:
# id->(Null values, not integer, duplicates);cast->(Null values, not array);crew->(Null values, not array)

# cast and crew columns are Stringified JSON format which we will parce into following columns:
# (character ,name, gender) -> for dim_staff table;   (job, department, name, gender) -> for dim_crew table.

[Row(cast="[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE

In [12]:
## Exploring data structures of ratings_df
# For our data analysis Team we are interested in the following column extraction:
# cast, crew, id

ratings_df.show(2)

#Following quality issues:
# movieId->(Null values, not integer, duplicates);rating->(Null values, not float);timestamp->(Null values, not timestamp)



+-------+------+----------+------+
|movieId|rating| timestamp|userId|
+-------+------+----------+------+
|movieId|rating| timestamp|userId|
|    110|   1.0|1425941529|     1|
+-------+------+----------+------+
only showing top 2 rows



In [13]:
#### Cleaning Steps

In [14]:

#Filtering movies dataframe
movies_df = movies_df.filter(col("id").cast("int").isNotNull())
movies_df = movies_df.filter(col("production_companies").startswith('['))
#Filtering keywords dataframe
keywords_df = keywords_df.filter(col("id").cast("int").isNotNull())
keywords_df = keywords_df.filter(col("keywords").startswith('[{'))

#Leaving only usefull columns in movies df 
movies_df = movies_df.select(col("id").alias("movie_key"),col("title"),col("overview"),col("release_date"),col("original_language").alias("language"),col("runtime"),col("vote_average"),col("production_companies"),col("vote_count"),col("revenue"),col("budget"))
#Left joining movies df with keywords df and dropping id column 
full_movies_df = movies_df.join(keywords_df,movies_df.movie_key ==  keywords_df.id,"left")
full_movies_df = full_movies_df.drop(full_movies_df.id)


In [15]:
full_movies_df = full_movies_df.withColumn("movie_key", full_movies_df["movie_key"].cast(IntegerType()))
full_movies_df = full_movies_df.withColumn("runtime", full_movies_df["runtime"].cast(IntegerType()))
full_movies_df = full_movies_df.withColumn("vote_average", full_movies_df["vote_average"].cast(DoubleType()))
full_movies_df = full_movies_df.withColumn("vote_count", full_movies_df["vote_count"].cast(DoubleType()))
full_movies_df = full_movies_df.withColumn("revenue", full_movies_df["revenue"].cast(DoubleType()))
full_movies_df = full_movies_df.withColumn("budget", full_movies_df["budget"].cast(DoubleType()))
full_movies_df.printSchema()
full_movies_df.show(5)

root
 |-- movie_key: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- language: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- vote_count: double (nullable = true)
 |-- revenue: double (nullable = true)
 |-- budget: double (nullable = true)
 |-- keywords: string (nullable = true)

+---------+--------------------+--------------------+------------+--------+-------+------------+--------------------+----------+------------+------+--------------------+
|movie_key|               title|            overview|release_date|language|runtime|vote_average|production_companies|vote_count|     revenue|budget|            keywords|
+---------+--------------------+--------------------+------------+--------+-------+------------+--------------------+----------+------------+------+--------

In [16]:
# Writing full_movies_df to parquet file in S3

full_movies_df.write.mode("overwrite").parquet('s3a://tempbucket1168/stage_movies.parquet')

In [17]:
#Creating and filtering movie_staff/movie_crew dataframes from  credits df
# creating and filtering movie_staff_df
movie_staff_df = credits_df.select(col("id"),col("cast")).filter(col("id").cast("int").isNotNull()).filter(col("cast").startswith('[{'))
movie_staff_df = movie_staff_df.select(col("id"),explode(from_json(col("cast"), ArrayType(StringType()))).alias("staff_info"))
movie_staff_df = movie_staff_df.select(col("id").alias("movie_id"),json_tuple(col("staff_info"),"character","name","gender").alias("character","actor_name","gender"))
movie_staff_df.show(5)

# creating and filtering movie_crew_df
movie_crew_df = credits_df.select(col("id"),col("crew")).filter(col("id").cast("int").isNotNull()).filter(col("crew").startswith('[{'))
movie_crew_df = movie_crew_df.select(col("id"),explode(from_json(col("crew"), ArrayType(StringType()))).alias("crew_info"))
movie_crew_df = movie_crew_df.select(col("id").alias("movie_id"),json_tuple(col("crew_info"),"job","department","name","gender").alias("job","department","name","gender"))
movie_crew_df.show(5)

+--------+--------------------+--------------+------+
|movie_id|           character|    actor_name|gender|
+--------+--------------------+--------------+------+
|   15602|         Max Goldman|Walter Matthau|     2|
|   15602|      John Gustafson|   Jack Lemmon|     2|
|   15602|     Ariel Gustafson|   Ann-Margret|     1|
|   15602|Maria Sophia Cole...|  Sophia Loren|     1|
|   15602|   Melanie Gustafson|  Daryl Hannah|     1|
+--------+--------------------+--------------+------+
only showing top 5 rows

+--------+------------+----------+-------------------+------+
|movie_id|         job|department|               name|gender|
+--------+------------+----------+-------------------+------+
|   16420|      Writer|   Writing|William Shakespeare|     2|
|   16420|    Director| Directing|      Oliver Parker|     2|
|   16420|  Adaptation|   Writing|      Oliver Parker|     2|
|   31174|    Director| Directing|  Richard Loncraine|     2|
|   31174|Theatre Play|   Writing|William Shakespeare| 

In [18]:
movie_staff_df = movie_staff_df.withColumn("movie_id", movie_staff_df["movie_id"].cast(IntegerType()))
movie_crew_df = movie_crew_df.withColumn("movie_id", movie_crew_df["movie_id"].cast(IntegerType()))

In [19]:
movie_staff_df.printSchema()
movie_crew_df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- character: string (nullable = true)
 |-- actor_name: string (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- movie_id: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- department: string (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



In [20]:
# Writing to movie_staff_df parquet file in S3
movie_staff_df.write.mode("overwrite").parquet('s3a://tempbucket1168/dim_movie_staff.parquet')


In [21]:
# Writing to parquet file
movie_crew_df.write.mode("overwrite").parquet('s3a://tempbucket1168/dim_movie_crew.parquet')

In [23]:
#Creating and filtering ratings_df dataframes
#Filtering data
ratings_df = ratings_df.withColumn("userId", ratings_df["userId"].cast(IntegerType()).isNotNull())
ratings_df = ratings_df.withColumn("movieId", ratings_df["movieId"].cast(IntegerType()).isNotNull())
ratings_df = ratings_df.withColumn("rating", ratings_df["rating"].cast(DoubleType()).isNotNull())
ratings_df = ratings_df.withColumn("timestamp", from_unixtime((col("timestamp")/1000),"yyyy-MM-dd HH:mm:ss").isNotNull())
ratings_df = ratings_df.withColumn("timestamp", ratings_df["timestamp"].cast(TimestampType()))

In [24]:
ratings_df.printSchema()

root
 |-- user_key: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [25]:
# Writing ratings_df to stage_ratings.parquet in S3
ratings_df.write.mode("overwrite").parquet('s3a://tempbucket1168/stage_ratings.parquet')

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
### Amazon Redshift - ERD

>  For Amazon Redshift Datawarehouse I have chosen Star Schema (see pic. below) of tables, which
> allows quick analysys of data by using simple queries without JOIN statements.
>![Alt text](./ERD_Capstone.PNG?raw=true "Title")


### Data Dictionary

**fact_movies** - _contains PRIMARY keys of dimentional tables and budget/revenue metrics of each movie_

|Field Name     |Datatype |Field length |Constraint  |Description     |
| ------------- | ------  | ----------- | ---------- | -------------- |
|fact_movie_key |INT      | 5           | PRIMARY KEY| fact movie id  |
|movie_key      |INT      | 5           | FOREIGN KEY|      movie id  |
|user_key       |INT      | 5           | FOREIGN KEY|      user id   |
|date_key       |TIMESTAMP| 5           | FOREIGN KEY|    timestamp   |
|movie_staff_key|INT      | 5           | FOREIGN KEY|movie staff id  |
|movie_crew_key |INT      | 5           | FOREIGN KEY|movie crew  id  |
|budget         |FLOAT    | 15          | NOT NULL   |movie budget    |
|revenue        |FLOAT    | 15          | NOT NULL   |movie revenue   |



**dim_movie** - _contains information of 45000 movies_

|Field Name          |Datatype |Field length |Constraint  |Description                  |
| ------------------ | ------  | ----------- | ---------- | ------------------          |
|movie_key           |INT      | 5           | PRIMARY KEY|  movie id                   |
|title               |VARCHAR  | 20          | NOT NULL   |   movie title               |
|overview            |VARCHAR  | 10000       | NOT NULL   | movie description           |
|release_date        |VARCHAR  | 20          | NOT NULL   |movie release date           |
|language            |VARCHAR  | 10          | NOT NULL   |    movie language           | 
|runtime             |INT      | 5           | NOT NULL   |movie length                 |
|vote_avg            |FLOAT    | 5           | NOT NULL   |movie's average vote         |
|vote_count          |FLOAT    | 15          | NOT NULL   | vote's count                |
|production_companies|VARCHAR  | 15          | NOT NULL   |movie's  production companies|
|keywords            |VARCHAR  | 15000       | NOT NULL   |movie's  keywords            |


**dim_users** - _contains information of 270000 users with 26000000 movie ratings_

|Field Name     |Datatype |Field length |Constraint  |Description     |
| ------------- | ------  | ----------- | ---------- | -------------- |
|user_key       |INT      | 5           | PRIMARY KEY|      user id   |
|movieID        |INT      | 5           | NOT NULL   |    movie id    |
|rating         |FLOAT    | 5           | NOT NULL   |movie's rating  |


**dim_date** - _date time-table when ratings were given_

|Field Name     |Datatype |Field length |Constraint  |Description     |
| ------------- | ------  | ----------- | ---------- | -------------- |
|date_key       |TIMESTAMP| 5           | PRIMARY KEY|    timestamp   |
|hour           |INT      | 5           | NOT NULL   |         hour   |
|day            |INT      | 5           | NOT NULL   |day             |
|week           |INT      | 5           | NOT NULL   |week            |
|month          |INT      | 5           | NOT NULL   |month           |
|year           |INT      | 5           | NOT NULL   |year            |
|weekday        |INT      | 5           | NOT NULL   |weekday         |


**dim_movie_staff** - _contains information of actors and roles, which they played in movies_

|Field Name     |Datatype |Field length |Constraint  |Description       |
| ------------- | ------  | ----------- | ---------- | ---------------- |
|movie_staff_key|INT      | 5           | PRIMARY KEY| movie staff id   |
|character      |VARCHAR  | 20          | NOT NULL   |movie character   |
|name           |VARCHAR  | 20          | NOT NULL   |character's name  |
|movie_id       |INT      | 5           | NOT NULL   |movie id          |
|gender         |VARCHAR  | 20          | NOT NULL   |character's gender|


**dim_movie_crew** - _contains information of crew of each movie_

|Field Name     |Datatype |Field length |Constraint  |Description       |
| ------------- | ------  | ----------- | ---------- | ---------------- |
|movie_crew_key |INT      | 5           | PRIMARY KEY| movie crew  id   |
|job            |VARCHAR  | 20          | NOT NULL   |crew's job        |
|department     |VARCHAR  | 20          | NOT NULL   |crew's  department|
|movie_id       |INT      | 5           | NOT NULL   |movie id          |
|name           |VARCHAR  | 20          | NOT NULL   |crew's name       |
|gender         |VARCHAR  | 20          | NOT NULL   |crew's      gender|


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Data pipeline is following:

![Alt text](./Pipeline_Capstone.PNG?raw=true "Title")

As it can be seen there are three major steps in the pipeline:

1. COPY data to staging/dim tables in AWS Redshift
2. INSERT data from staging tables to fact/dim tables
3. RUN data quality check


#### 4.3 Running the Project 
To run this project Apache Airflow should be installed.

- Dags and plugins files should be copied to the Airflow Environment.
- Setup connection for aws_credentials with aws_access_key_id and aws_secret_access_key given in  Admin pillar
- Setup connection for aws_redshift with password, host, schema given in  Admin pillar
- After connections set -> turn on DAG in Apache Airflow UI

#### Step 5: Complete Project Write Up

The following scenaurious should be forecasted and prepared:

**The data was increased by 100x.**  Amazon Redshift's computing capacity(Cluster) can be adjusted at any time. This is the main cloud concept, which makes it flexible to adjust for any amount of data.

**The pipelines would be run on a daily basis by 7 am every day.** Thank Apache Airflow, it can be easilz adjusted in Airflow DAG definitions.

**The database needed to be accessed by 100+ people.** As described above, Amazon Redshift is highly flexible in terms of capacity adjustment.


### Query Results
> **Query to analyse movies with revenue > 1000000**
>
> SELECT DISTINCT dim_movies.title, fact_movies.revenue, dim_movies.vote_average
> FROM fact_movies
> LEFT OUTER JOIN dim_movies ON (fact_movies.movie_key = dim_movies.movie_key)
> WHERE  fact_movies.revenue > 1000000
> ORDER BY  fact_movies.revenue DESC
>
> ![Alt text](./Query_Redshift1.PNG?raw=true "Title")

> **Query to analyse movie with revenue > 1000000 and average_vote == 7.0**
>
> SELECT DISTINCT dim_movies.title, dim_movies.vote_average, dim_movie_staff.actor_name, dim_movies.keywords
> FROM fact_movies 
> LEFT OUTER JOIN dim_movies ON (fact_movies.movie_key = dim_movies.movie_key)
> LEFT OUTER JOIN dim_movie_staff ON (fact_movies.movie_key = dim_movie_staff.movie_id)
> WHERE dim_movies.title = 'House of Games'
>
> ![Alt text](./Query_Redshift2.PNG?raw=true "Title")
